In [8]:
import numpy as np
import scipy.stats
from tqdm import tqdm

## GloRiC Attribution

In [9]:
arcpy.env.workspace = r"C:\Users\olivoshj\Desktop\ArcGIS_Projects\Dam_Project_Aux\Dam_Project_Aux.gdb"
input_ranges = "Global_Grid"
# downloaded from https://www.hydrosheds.org/products/gloric
gloric = "GloRiC_EE"

In [10]:
arcpy.management.AddFields(input_ranges, "CMI_indx FLOAT # # #; Class_count LONG # # #; Class_geom SHORT # # #;Class_hydr SHORT # # #;Class_phys SHORT # # #;Lake_wet FLOAT # # #;Log_Q_avg FLOAT # # #;Log_Q_var FLOAT # # #;Reach_type SHORT # # #;Stream_pow FLOAT # # #;Temp_min FLOAT # # # #;GloRiC_km FLOAT # # #;")

<Result 'Global_Grid'>

In [ ]:
## Iterates through each range calculating GloRiC stats
total = int(arcpy.management.GetCount(input_ranges)[0])
fields = ["SHAPE@","CMI_indx","Class_count","Class_geom","Class_hydr","Class_phys","Lake_wet","Log_Q_avg","Log_Q_var","Reach_type","Stream_pow","Temp_min", "GloRiC_km", "OBJECTID"]
# row ids     0        1             2             3            4            5          6           7          8             9           10           11         12          13   

with arcpy.da.UpdateCursor(input_ranges, fields) as cursor:
    for row in tqdm(cursor, total=total):
        # clips GloRiC hydrography for each range
        arcpy.analysis.PairwiseClip(
            in_features=gloric,
            clip_features=row[0],
            out_feature_class="currLines"
        )

        if int(arcpy.management.GetCount("currLines")[0]) == 0:
            continue
        
        row[1] = np.mean([r[0] for r in arcpy.da.SearchCursor("currLines","CMI_indx",where_clause="CMI_indx IS NOT NULL")])
        row[2] = len(list(set([r[0] for r in arcpy.da.SearchCursor("currLines",'Reach_type')])))
        row[3] = scipy.stats.mode([r[0] for r in arcpy.da.SearchCursor("currLines",'Class_geom',where_clause="Class_geom IS NOT NULL")],keepdims=False)[0]
        row[4] = scipy.stats.mode([r[0] for r in arcpy.da.SearchCursor("currLines",'Class_hydr',where_clause="Class_hydr IS NOT NULL")],keepdims=False)[0]
        row[5] = scipy.stats.mode([r[0] for r in arcpy.da.SearchCursor("currLines",'Class_phys',where_clause="Class_phys IS NOT NULL")],keepdims=False)[0]
        row[6] = np.sum([r[0] for r in arcpy.da.SearchCursor("currLines",'Lake_wet',where_clause="Lake_wet IS NOT NULL")])
        row[7] = np.mean([r[0] for r in arcpy.da.SearchCursor("currLines",'Log_Q_avg', where_clause="Log_Q_avg IS NOT NULL")])
        row[8] = np.mean([r[0] for r in arcpy.da.SearchCursor("currLines",'Log_Q_var', where_clause="Log_Q_var IS NOT NULL")])
        row[9] = scipy.stats.mode([r[0] for r in arcpy.da.SearchCursor("currLines",'Reach_type',where_clause="Reach_type IS NOT NULL")],keepdims=False)[0]
        row[10] = np.mean([r[0] for r in arcpy.da.SearchCursor("currLines",'Stream_pow', where_clause="Stream_pow IS NOT NULL")])
        row[11] = np.mean([r[0] for r in arcpy.da.SearchCursor("currLines",'Temp_min', where_clause="Temp_min IS NOT NULL")])
        row[12] = np.sum([r[0] for r in arcpy.da.SearchCursor("currLines",'Length_km', where_clause="Length_km IS NOT NULL")])

        cursor.updateRow(row)
        
arcpy.management.Delete(fr"{arcpy.env.workspace}\currLines")

  1%|          | 180/16528 [12:59<19:11:44,  4.23s/it]C:\Users\olivoshj\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-3\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\olivoshj\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone-3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
 98%|█████████▊| 16126/16528 [29:08:58<1:15:41, 11.30s/it] 

In [ ]:
arcpy.conversion.ExportTable(
    in_table=input_ranges,
    out_table=r"R:\FWL\Arismendi-Lab\Andres\Gilbert_Freshwater_Fish_Analysis\Revised_Analysis_NatureCommunications\Input_datasets\GloRiC_Attributes.csv",
    field_mapping=f'sci_name "sci_name" true true false 100 Text 0 0,First,#,{input_ranges},sci_name,0,99;CMI_indx "CMI_indx" true true false 4 Float 0 0,First,#,{input_ranges},CMI_indx,-1,-1;Class_count "Class_count" true true false 4 Long 0 0,First,#,{input_ranges},Class_count,-1,-1;Class_geom "Class_geom" true true false 2 Short 0 0,First,#,{input_ranges},Class_geom,-1,-1;Class_hydr "Class_hydr" true true false 2 Short 0 0,First,#,{input_ranges},Class_hydr,-1,-1;Class_phys "Class_phys" true true false 2 Short 0 0,First,#,{input_ranges},Class_phys,-1,-1;Lake_wet "Lake_wet" true true false 4 Float 0 0,First,#,{input_ranges},Lake_wet,-1,-1;Log_Q_avg "Log_Q_avg" true true false 4 Float 0 0,First,#,{input_ranges},Log_Q_avg,-1,-1;Log_Q_var "Log_Q_var" true true false 4 Float 0 0,First,#,{input_ranges},Log_Q_var,-1,-1;Reach_type "Reach_type" true true false 2 Short 0 0,First,#,{input_ranges},Reach_type,-1,-1;Stream_pow "Stream_pow" true true false 4 Float 0 0,First,#,{input_ranges},Stream_pow,-1,-1;Temp_min "Temp_min" true true false 4 Float 0 0,First,#,{input_ranges},Temp_min,-1,-1;GloRiC_km "GloRiC_km" true true false 4 Float 0 0,First,#,{input_ranges},GloRiC_km,-1,-1',
    where_clause="GloRiC_km IS NOT NULL"
)